In [1]:
from llama_index.core.llama_dataset.rag import LabelledRagDataset

dataset_json = "./data/output/test-dataset.json"

dataset = LabelledRagDataset.from_json(dataset_json)
examples = dataset.examples

In [3]:
examples[:2]

[LabelledRagDataExample(query="E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상 직종)'에 대해 설명하시오.", query_by=CreatedBy(model_name='gpt-4o', type=<CreatedByType.AI: 'ai'>), reference_contexts=["[**E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상\n직종)'이란?\n2**](#e-7-사증-소지자들이-취업할-수-있는-분야인-도입직종발급-대상-직종이란)  \n[**E-7 '도입직종(발급 대상 직종)'의 유형\n3**](#e-7-도입직종발급-대상-직종의-유형)  \n[**E-7 '도입직종(발급 대상 직종)'의 분류,"], reference_answer="E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상 직종)'에 대해 설명하자면, 이는 E-7 비자를 소지한 외국인들이 한국에서 취업할 수 있는 특정 직종을 의미합니다. '도입직종'은 E-7 비자를 발급받을 수 있는 직종을 지칭하며, 이러한 직종은 한국 정부가 정한 기준에 따라 분류되고 관리됩니다. E-7 비자는 주로 전문직, 기술직, 특정 산업 분야에서 필요한 인력을 대상으로 발급되며, 이를 통해 외국인들이 한국에서 합법적으로 근무할 수 있게 됩니다.", reference_answer_by=CreatedBy(model_name='gpt-4o', type=<CreatedByType.AI: 'ai'>)),
 LabelledRagDataExample(query='E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대해 설명하시오.', query_by=CreatedBy(model_name='gpt-4o', type=<CreatedByType.AI: 'ai'>), reference_contexts=["2**](#e-7-사증-소지자들이-취업할-수-있는-분야인-도입직종발급-대상-직종이란)  \n[**E-7 '도입직종(발급 대상 직종)'의 유형\n3**](#e-7

In [68]:
from llama_index.core.evaluation import AnswerRelevancyEvaluator
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter

question = examples[0].query

node_parser = SentenceSplitter()
nodes = node_parser.get_nodes_from_documents(documents)

llm = OpenAI(model="gpt-4o", temperature=0)

Settings.llm = OpenAI(model="gpt-4o", temperature=0)
vector_index = VectorStoreIndex(nodes)
engine = vector_index.as_query_engine()
response = engine.query(question)
answer = str(response)

print(f"{question}")
print(f"Answer: {answer}")
evaluator = AnswerRelevancyEvaluator(llm)
result = evaluator.evaluate(query=question, response=answer)
print(f"score: {result.score}")
print(f"feedback: {result.feedback}")

# Display results
# Question: How did Ultron initially come into existence and what was his ultimate goal?
# Answer: Ultron initially came into existence when Tony Stark and Bruce Banner discovered an artificial intelligence within Loki's scepter and decided to use it to complete Stark's "Ultron" global defense program. Ultron's ultimate goal was to eradicate humanity in order to save Earth.
# score: 1.0
# feedback: 1. The provided response matches the subject matter of the user's query by explaining how
#  Ultron initially came into existence and what his ultimate goal was.
# 2. The response directly addresses the focus and perspective of the user's query by detailing the specific events that led to Ultron's creation and his ultimate goal of eradicating humanity.
# [RESULT] 2

E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상 직종)'에 대해 설명하시오.
Answer: 'E-7 사증 소지자들이 취업할 수 있는 분야인 '도입직종(발급 대상 직종)'은 법무부장관이 국가경쟁력 강화를 위해 전문적인 지식, 기술 또는 기능을 가진 외국인력 도입이 특히 필요하다고 지정한 분야를 말합니다. 국가 및 지방자치단체는 이러한 외국인력을 유치하고 그들의 법적 지위 및 처우를 개선하기 위한 제도와 시책을 마련하기 위해 노력해야 합니다.
score: 1.0
feedback: 1. 제공된 응답이 사용자의 질문 주제와 일치합니까?
네, 제공된 응답은 사용자의 질문 주제인 'E-7 사증 소지자들이 취업할 수 있는 분야인 도입직종(발급 대상 직종)'에 대해 설명하고 있습니다.

2. 제공된 응답이 사용자의 질문에서 다루고자 하는 초점이나 관점을 해결하려고 시도합니까?
네, 제공된 응답은 '도입직종(발급 대상 직종)'이 무엇인지, 그리고 그 분야가 어떻게 지정되는지에 대해 설명하고 있습니다. 또한, 국가 및 지방자치단체의 역할에 대해서도 언급하고 있어 사용자의 질문에 대한 초점을 잘 맞추고 있습니다.

[RESULT] 2


In [70]:
from llama_index.core.evaluation.answer_relevancy import DEFAULT_EVAL_TEMPLATE

translate_prompt = "\n\nPlease reply in Korean."
eval_template = DEFAULT_EVAL_TEMPLATE
eval_template.template += translate_prompt
evaluator = AnswerRelevancyEvaluator(
    llm=llm, eval_template=eval_template
)


question = examples[1].query

response = engine.query(question)
answer = str(response)

print(f"Question: {question}")
print(f"Answer: {answer}")
evaluator = AnswerRelevancyEvaluator(llm)
result = evaluator.evaluate(query=question, response=answer)
print(f"score: {result.score}")
print(f"feedback: {result.feedback}")

Question: E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대해 설명하시오.
Answer: E-7 사증 발급 신청 시 필요한 첨부서류는 다음과 같습니다:

1. 피초청인(외국인) 준비서류:
   - 여권사본
   - 반명함판 칼라사진 1매
   - 고용계약서
   - 자격요건 입증서류 (학위증, 경력증명서, 자격증 등)
   - 국외에서 발급한 서류는 국문 또는 영문 번역본 첨부, 주요 서류는 영사 공증 또는 아포스티유 확인서 제출

2. 초청인 준비서류:
   - 고용단체 등 설립관련서류
   - 외국인 고용의 필요성을 입증할 수 있는 서류 (초청사유서, 고용추천서 등)
   - 신원보증서 (법무부장관이 고시한 근무처변경, 추가 신고가 제한되는 직종 종사자만 해당)
   - 납세증명서 (국세완납증명서)
   - 지방세 납세증명

신청 절차는 초청업체 소재지 관할 출입국ㆍ외국인청에 신청하며, 주무부처 등의 우수인재 고용추천을 받은 경우 온라인으로도 신청이 가능합니다. KOTRA/중진공의 골드카드 고용추천을 받은 경우 온라인 신청 시 고용추천서 외 첨부서류는 면제되며, 나머지 서류는 외국인등록 시 제출합니다.
score: 1.0
feedback: 피드백:

1. 제공된 응답이 사용자의 질문 주제와 일치하는가?
   - 네, 사용자의 질문은 E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대한 설명을 요청하고 있습니다. 제공된 응답은 E-7 사증 발급 신청 시 필요한 첨부서류와 신청 절차에 대해 상세히 설명하고 있습니다.

2. 제공된 응답이 사용자의 질문에서 다루고자 하는 초점이나 관점을 시도하고 있는가?
   - 네, 제공된 응답은 사용자가 요청한 첨부서류 목록과 신청 절차를 구체적으로 다루고 있습니다. 피초청인과 초청인의 준비서류를 구분하여 설명하고 있으며, 신청 절차에 대해서도 명확히 안내하고 있습니다.

[RESULT] 2


In [71]:
from llama_index.core.evaluation.context_relevancy import DEFAULT_EVAL_TEMPLATE
from llama_index.core.evaluation import ContextRelevancyEvaluator

translate_prompt = "\n\nPlease reply in Korean."
eval_template = DEFAULT_EVAL_TEMPLATE
eval_template.template += translate_prompt

contexts = [n.get_content() for n in response.source_nodes]
evaluator = ContextRelevancyEvaluator(llm, eval_template=eval_template)
result = evaluator.evaluate(query=question, contexts=contexts)

print(f"Question: {question}")
print(f"score: {result.score}")
print(f"feedback: {result.feedback}")

Question: E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대해 설명하시오.
score: 1.0
feedback: ### 피드백

1. **Does the retrieved context match the subject matter of the user's query?**
   - **점수: 2/2**
     - 사용자의 질문은 E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대한 설명을 요구하고 있습니다. 제공된 문맥은 E-7 사증 발급 신청 시 필요한 첨부서류와 신청 절차에 대해 상세히 설명하고 있습니다. 따라서 주제와 완벽히 일치합니다.

2. **Can the retrieved context be used exclusively to provide a full answer to the user's query?**
   - **점수: 2/2**
     - 제공된 문맥은 E-7 사증 발급 신청 시 필요한 첨부서류와 신청 절차에 대해 매우 구체적이고 포괄적인 정보를 제공합니다. 피초청인과 초청인이 준비해야 할 서류 목록, 신청 절차, 온라인 신청 방법 등 모든 필요한 정보를 포함하고 있어, 이 문맥만으로도 사용자의 질문에 완전한 답변을 제공할 수 있습니다.

### 최종 결과
[RESULT] 4.0


In [74]:
from llama_index.core.evaluation.relevancy import DEFAULT_EVAL_TEMPLATE
from llama_index.core.evaluation import RelevancyEvaluator

translate_prompt = "\n\nPlease reply in Korean."
eval_template = DEFAULT_EVAL_TEMPLATE
eval_template.template += translate_prompt


evaluator = RelevancyEvaluator(llm, eval_template=eval_template)
result = evaluator.evaluate(query=question, response=answer, contexts=contexts)
print(f"score: {result.score}")
print(f"feedback: {result.feedback}")
print(f"passing: {result.passing}")

# Display results
# score: 1.0
# feedback: YES
# passing: True

score: 1.0
feedback: YES
passing: True


In [75]:
from llama_index.core.evaluation.faithfulness import DEFAULT_EVAL_TEMPLATE
from llama_index.core.evaluation import FaithfulnessEvaluator

translate_prompt = "\n\nPlease reply in Korean."
eval_template = DEFAULT_EVAL_TEMPLATE
eval_template.template += translate_prompt

evaluator = FaithfulnessEvaluator(llm, eval_template=eval_template)
result = evaluator.evaluate(response=answer, contexts=contexts)
print(f"score: {result.score}")
print(f"feedback: {result.feedback}")
print(f"passing: {result.passing}")

# Display results
# score: 1.0
# feedback: YES
# passing: True

score: 1.0
feedback: YES
passing: True


In [86]:
from llama_index.core.evaluation.correctness import DEFAULT_EVAL_TEMPLATE
from llama_index.core.evaluation import CorrectnessEvaluator


# translate_prompt = "\n\nPlease reply in Korean."
# eval_template = DEFAULT_EVAL_TEMPLATE
# eval_template.template += translate_prompt

evaluator = CorrectnessEvaluator(llm)
ground_truth = examples[1].reference_answer
print(f"{question}")
print(f"Answer: {answer}")
print(f"Ground Truth: {ground_truth}")
result = evaluator.evaluate(query=question, response=answer, reference=ground_truth)
print(f"score: {result.score}")
print(f"feedback: {result.feedback}")
print(f"passing: {result.passing}")

E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대해 설명하시오.
Answer: E-7 사증 발급 신청 시 필요한 첨부서류는 다음과 같습니다:

1. 피초청인(외국인) 준비서류:
   - 여권사본
   - 반명함판 칼라사진 1매
   - 고용계약서
   - 자격요건 입증서류 (학위증, 경력증명서, 자격증 등)
   - 국외에서 발급한 서류는 국문 또는 영문 번역본 첨부, 주요 서류는 영사 공증 또는 아포스티유 확인서 제출

2. 초청인 준비서류:
   - 고용단체 등 설립관련서류
   - 외국인 고용의 필요성을 입증할 수 있는 서류 (초청사유서, 고용추천서 등)
   - 신원보증서 (법무부장관이 고시한 근무처변경, 추가 신고가 제한되는 직종 종사자만 해당)
   - 납세증명서 (국세완납증명서)
   - 지방세 납세증명

신청 절차는 초청업체 소재지 관할 출입국ㆍ외국인청에 신청하며, 주무부처 등의 우수인재 고용추천을 받은 경우 온라인으로도 신청이 가능합니다. KOTRA/중진공의 골드카드 고용추천을 받은 경우 온라인 신청 시 고용추천서 외 첨부서류는 면제되며, 나머지 서류는 외국인등록 시 제출합니다.
Ground Truth: E-7 사증 발급 신청 시 필요한 첨부서류 및 신청 절차에 대한 정보는 아래와 같습니다.

### 첨부서류
1. **사증발급신청서**: 신청자가 작성하여 제출해야 합니다.
2. **여권**: 유효한 여권 원본과 사본을 제출해야 합니다.
3. **사진**: 최근 6개월 이내에 촬영한 여권용 사진을 제출해야 합니다.
4. **고용계약서**: 고용주와 체결한 고용계약서 사본을 제출해야 합니다.
5. **학력 및 경력 증명서**: 해당 직종에 필요한 학력 및 경력을 증명할 수 있는 서류를 제출해야 합니다.
6. **고용추천서**: 필요 시 고용주가 작성한 고용추천서를 제출해야 합니다.
7. **기타 서류**: 해당 직종 및 신청자의 상황에 따라 추가로 요구될 수 있는 서류들이 있을 수 있습니다.

### 신청 절차
1. **서류 준비**: